In [10]:
import numpy as np
import pandas as pd
import matplotlib as plt
import json
import warnings

# Data manipulation
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics 

# Modeling code
from xgboost.sklearn import XGBClassifier #XGBoost


# Over Sampling code isn't really needed for our data set, but included as a reference for fraud detection
from imblearn.over_sampling import SMOTE

In [12]:
'''
Gotta fix this, but ignore for now:

<ipython-input-9-098e2ff8e245>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samples['DC'] = np.where(df['publisher'] == 'DC Comics', True, False)
<ipython-input-9-098e2ff8e245>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead
'''

from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [13]:
df = pd.read_json('samples.json')
# TODO, Avoid the traditional df.head(10) here because the columns are too long
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103086 entries, 0 to 103085
Data columns (total 21 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   comic                   103086 non-null  object 
 1   location                103086 non-null  object 
 2   pages                   103086 non-null  int64  
 3   match                   103086 non-null  bool   
 4   distance                103086 non-null  float64
 5   id                      103086 non-null  int64  
 6   hash                    103086 non-null  object 
 7   name                    73329 non-null   object 
 8   description             84187 non-null   object 
 9   image                   103086 non-null  object 
 10  month                   103086 non-null  object 
 11  issue_number            103083 non-null  object 
 12  url                     103086 non-null  object 
 13  cover_date              103086 non-null  object 
 14  volume_id           

In [5]:
samples = df[['comic', 'pages', 'distance', 'volume_count_of_issues', 'issue_number', 'publisher', 'match']]
samples.head(10)

,comic,pages,distance,volume_count_of_issues,issue_number,publisher,match
0,New X-Men #127 (2002-08-01) - Of Living and Dy...,21,0.6250,28.0,8,Panini Comics,False
1,New X-Men #127 (2002-08-01) - Of Living and Dy...,21,0.6250,25.0,13,Cómics Forum,False
2,New X-Men #127 (2002-08-01) - Of Living and Dy...,21,0.5625,12.0,111,Panini Comics,False
3,New X-Men #127 (2002-08-01) - Of Living and Dy...,21,0.5625,365.0,153,Panini Comics,False
4,New X-Men #127 (2002-08-01) - Of Living and Dy...,21,0.1875,43.0,127,Marvel,True
5,Stormwatch #001 (2011-11-27) - The Dark Side P...,36,0.6250,1.0,1,UDON,False
6,Stormwatch #001 (2011-11-27) - The Dark Side P...,36,0.6250,4.0,2,Dark Horse Comics,False
7,Stormwatch #001 (2011-11-27) - The Dark Side P...,36,0.6250,7.0,6,Top Cow,False
8,Stormwatch #001 (2011-11-27) - The Dark Side P...,36,0.5000,1.0,1,DC Comics,False
9,Stormwatch #001 (2011-11-27) - The Dark Side P...,36,0.0625,31.0,1,DC Comics,True


In [18]:
# Implement one hot encoding for the publishers (DC, Marvel, Other)
samples['DC'] = np.where(df['publisher'] == 'DC Comics', True, False)
samples['Marvel'] = np.where(df['publisher'] == 'Marvel', True, False)
samples['first_issue'] = np.where(df['issue_number'] == '1', True, False)

# Convert pages column to integer data
samples['pages'] = pd.to_numeric(df['pages'])
samples.head(10)

,comic,pages,distance,volume_count_of_issues,issue_number,publisher,match,DC,Marvel,first_issue
0,New X-Men #127 (2002-08-01) - Of Living and Dy...,21,0.6250,28.0,8,Panini Comics,False,False,False,False
1,New X-Men #127 (2002-08-01) - Of Living and Dy...,21,0.6250,25.0,13,Cómics Forum,False,False,False,False
2,New X-Men #127 (2002-08-01) - Of Living and Dy...,21,0.5625,12.0,111,Panini Comics,False,False,False,False
3,New X-Men #127 (2002-08-01) - Of Living and Dy...,21,0.5625,365.0,153,Panini Comics,False,False,False,False
4,New X-Men #127 (2002-08-01) - Of Living and Dy...,21,0.1875,43.0,127,Marvel,True,False,True,False
5,Stormwatch #001 (2011-11-27) - The Dark Side P...,36,0.6250,1.0,1,UDON,False,False,False,True
6,Stormwatch #001 (2011-11-27) - The Dark Side P...,36,0.6250,4.0,2,Dark Horse Comics,False,False,False,False
7,Stormwatch #001 (2011-11-27) - The Dark Side P...,36,0.6250,7.0,6,Top Cow,False,False,False,False
8,Stormwatch #001 (2011-11-27) - The Dark Side P...,36,0.5000,1.0,1,DC Comics,False,True,False,True
9,Stormwatch #001 (2011-11-27) - The Dark Side P...,36,0.0625,31.0,1,DC Comics,True,True,False,True


In [21]:
# Looks good. Let's check the data types
samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103086 entries, 0 to 103085
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   comic                   103086 non-null  object 
 1   pages                   103086 non-null  int64  
 2   distance                103086 non-null  float64
 3   volume_count_of_issues  103083 non-null  float64
 4   issue_number            103083 non-null  object 
 5   publisher               103083 non-null  object 
 6   match                   103086 non-null  bool   
 7   DC                      103086 non-null  bool   
 8   Marvel                  103086 non-null  bool   
 9   first_issue             103086 non-null  bool   
dtypes: bool(4), float64(2), int64(1), object(3)
memory usage: 5.1+ MB


In [25]:
# Break the data into test, train partitions
X = samples[['pages', 'distance', 'volume_count_of_issues', 'first_issue', 'DC', 'Marvel']]
y = samples[['match']]
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

In [29]:
# Check the output (X)
X_train.head()

,pages,distance,volume_count_of_issues,first_issue,DC,Marvel
58593,23,0.5625,50.0,False,True,False
61763,25,0.6875,8.0,False,False,False
79502,23,0.6875,67.0,False,False,False
36521,68,0.5000,104.0,False,True,False
75125,119,0.6875,21.0,False,False,True


In [31]:
# And.. Y
y_train.head()

,match
58593,False
61763,False
79502,False
36521,True
75125,False


In [24]:
# xgb = XGBClassifier(min_child_weight = 5, max_depth=12, learning_rate= 0.1, gamma= 0.2, colsample_bytree= 0.7)

# xgb.fit(X_train_sup, y_train)

NameError: name 'X_train_sup' is not defined